https://stepik.org/lesson/1576202/step/7

In [ ]:
# @title 🔧 Установка модуля и загрузка словаря в файл и переменную
!pip install navec
import requests
from navec import Navec

url_navec_news = 'https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar'

response = requests.get(url_navec_news, stream=True)
with open('navec_hudlit_v1_12B_500K_300d_100q.tar', 'wb') as f:
    f.write(response.content)

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
global_navec = Navec.load(path)

In [ ]:
# Проверка работы словаря
is_word = 'машина' in global_navec # проверка наличия слова в словаре
v = global_navec['каждый'] # embedding слова "каждый"
indx = global_navec.vocab['каждый'] # индекс слова "каждый" в словаре
w = global_navec.vocab.words[indx] # слово по индексу
is_word, v.shape, indx, w

(True, (300,), 161025, 'каждый')

In [ ]:
# Глобальная переменная для тестировввания
_global_var_text = [
"Как я отмечал во введении простейшая НС представляет собой",
"Это классический пример нейро сети",
"Каждая связь между нейронами имеет определенный"
]

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

# сюда копируйте класс WordsDataset, созданный на предыдущем занятии
class WordsDataset(data.Dataset):
    def __init__(self, prev_words=4):
        super().__init__()
        self.text = _global_var_text
        df = []
        for string in self.text:
            s = [word for word in string.lower().split()]
            for i in range(prev_words, len(s)):
                df.append(s[i-prev_words:i+1])
        self.df = df
        self.length = len(self.df)

    def __len__(self):
        return self.length

    def __getitem__(self, indx):
            prev_words = self.df[indx][:-1]
            prev_words_emb = torch.stack([torch.as_tensor(global_navec[word]) for word in prev_words])
            next_word = self.df[indx][-1]
            next_word_indx = global_navec.vocab[next_word]

            return prev_words_emb, torch.tensor(next_word_indx)

# здесь объявляйте класс модели
class Words_Model(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()

        self.rnn = nn.RNN(in_features, 16, batch_first=True)
        self.linear = nn.Linear(32, out_features, bias=True)

    def forward(self, x):
        # x = self.emb(x)
        x, _ = self.rnn(x)
        x = torch.cat((x[:, -1, :], x[:, 0, :]), dim=-1)
        return self.linear(x)

# сюда копируйте объекты d_train и train_data, созданные на предыдущем занятии
d_train = WordsDataset(prev_words=4)

train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

'''ЗДЕСЬ КЛЮЧЕВЫЕ РАСХОЖДЕНИЯ С ЭТАЛОННЫМ РЕШЕНИЕМ'''
# model = WordsRNN(100, len(global_navec.vocab)) # строка из эталонного решения
model = Words_Model(300, len(global_navec.vocab.words))

optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

epochs = 1 # число эпох обучения (в реальности нужно от 100 и более)
model.train()

for _e in range(epochs):
    # с помощью цикла for переберите батчи из объекта train_data
    for x_train, y_train in train_data:
        predict = model(x_train).squeeze()
        loss = loss_func(predict, y_train)

        # выполните один шаг обучения (градиентного спуска)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# переведите модель в режим эксплуатации
model.eval()
predict = "Такими были первые нейронные сети предложенные".lower().split()
total = 10 # число прогнозируемых слов (дополнительно к начальной фразе)

for _ in range(total):
    emb = torch.stack([torch.as_tensor(global_navec[word]) for word in predict[-4:]])
    indx = model(emb.unsqueeze(0)).argmax().item() # unsqueeze(0) для batch size
    # print(indx) # глянуть на индекс
    predict.append(global_navec.vocab.words[indx])

# выведите полученную строку на экран
predict = " ".join(predict)
print(predict)

# ['такими', 'были', 'первые', 'нейронные', 'сети', 'предложенные', 'ukraina', 'заметивший', 'эфирные', 'tape', 'анортосис', 'zmar', 'небезопасное', 'социально-экономическими', 'шиловой', 'подлинной']

такими были первые нейронные сети предложенные разный скептических матовая недоверию амита отомщена mag питерc обидим выручают


In [ ]:
# https://github.com/selfedu-rus/neuro-pytorch/blob/main/solves/4.5.5
# @title Проверка работоспособости эталонного решения
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim


class WordsDataset(data.Dataset):
    def __init__(self, navec_emb, prev_words=4):
        self.prev_words = prev_words
        self.navec_emb = navec_emb

        self.lines = _global_var_text
        self.vocab = set((" ".join(self.lines)).lower().split())
        self.vocab_size = len(self.vocab)

        data = []
        targets = []

        for t in self.lines:
            words = t.lower().split()
            for item in range(len(words)-self.prev_words):
                data.append([self.navec_emb[words[x]].tolist() for x in range(item, item + self.prev_words)])
                targets.append(self.navec_emb.vocab[words[item+self.prev_words]])

        self.data = torch.tensor(data)
        self.targets = torch.tensor(targets)

        self.length = len(data)

    def __getitem__(self, item):
        return self.data[item], self.targets[item]

    def __len__(self):
        return self.length


class WordsRNN(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.hidden_size = 16
        self.in_features = in_features
        self.out_features = out_features

        self.rnn = nn.RNN(in_features, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, out_features)

    def forward(self, x):
        x, h = self.rnn(x)
        y = self.out(h)
        return y


d_train = WordsDataset(global_navec)
train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

int_to_word = dict(enumerate((global_navec.vocab)))

# Здесь меняем:
# model = WordsRNN(100, len(global_navec.vocab)) # эталонное решение
model = WordsRNN(300, len(int_to_word))

optimizer = optim.Adam(params=model.parameters(), lr=0.01, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

epochs = 1
model.train()

for _e in range(epochs):
    for x_train, y_train in train_data:
        predict = model(x_train).squeeze(0)
        loss = loss_func(predict, y_train.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()
predict = "Такими были первые нейронные сети предложенные".lower().split()
total = 10

for _ in range(total):
    _data = torch.tensor([d_train.navec_emb[predict[-x]].tolist() for x in range(d_train.prev_words, 0, -1)])
    with torch.no_grad():
        p = model(_data.unsqueeze(0)).squeeze(0)
    indx = torch.argmax(p, dim=1)
    predict.append(int_to_word[indx.item()])

predict = " ".join(predict)
print(predict)

TypeError: object of type 'Vocab' has no len()